In [1]:
%matplotlib widget
from lib import get_tournament_list, get_season_list, ShotDistribution
import ipywidgets as widgets
from IPython.display import display
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import yaml
import pandas as pd

with open("./configs/course_coords.yml", "r") as f:
    course_points = yaml.safe_load(f)

In [2]:
tourns = get_tournament_list()
seasons = get_season_list()
strokes=[1, 2, 3, 4, 5, 'all', 'drive', 'approach']
scores=[None, 'sub', 'par', 'over', 1, 2, 3, 4, 5, 6, 7, 8, 'all']
holes=[i+1 for i in range(18)]
rounds=[1, 2, 3, 4, 'all']
flags=[1, 2, 3, 4, 'all']

plt.ioff()
mpl = plt.figure(tight_layout=True)
ax = mpl.add_subplot(111)
mpl.canvas.toolbar_visible = True
mpl.canvas.header_visible = False # Hide the Figure name at the top of the figure
mpl.canvas.footer_visible = False
mpl.canvas.resizable = True
mpl.canvas.capture_scroll = False
plt.ion()

pty = go.Figure()
pty.update_layout(
    mapbox_style="white-bg",
    mapbox_layers=[
        {
            "below": 'traces',
            "sourcetype": "raster",
            "sourceattribution": "United States Geological Survey",
            "source": [
                "https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"
            ]
        },
    ],
    mapbox=dict(
        center=go.layout.mapbox.Center(
        lat=33.64,
        lon=-111.912,
        ),
        zoom=16,
    ),
    margin={"r":0,"t":0,"l":0,"b":0},
)

s = ShotDistribution()
use_plotly = False

In [3]:
def get_tourn_data(Tournament, Season):
    global s, use_plotly, quick_render
    try:
        if Season == 'all':
            s.generate_distribution(Tournament, course_points["COURSE_POINTS"])
        else:
            s.generate_distribution(Tournament, course_points["COURSE_POINTS"], Season)
        use_plotly=True
        quick_render.value = True
        quick_render.disabled = False
    except Exception as e:
        print(e)
        print("Entering calibration mode.")
        s.generate_distribution(Tournament)
        quick_render.value = True
        quick_render.disabled = True
        
tourn_data = widgets.interactive(get_tourn_data, Tournament=tourns, Season=seasons)

In [13]:
graph_out = widgets.Output(layout = {
            'width': '80%',
            'height': '600px',
            'border': '1px solid black'
        })

img_out = widgets.Output(layout = {
            'width': '100%',
#             'height': '400px',
            'border': '1px solid black'
        })

score_out = widgets.Output(layout = {
#             'width': '100%',
#             'height': '600px',
            'border': '1px solid black'
        })

def get_hole_data(Hole, Stroke, Round, Flag_Loc, Score1, Score2, Score3, Include_Shots, Include_Flag_Locs, Include_Flag_Prox, Include_Spray, Quick_Render):
    global use_plotly, quick_render
    global ax, pty
        
    plot_bool = use_plotly and (not quick_render.value)
            
    with score_out:
        try:
            df = s.as_df(Hole, None, None, None, flag_loc=Flag_Loc)
            score_avg = pd.pivot_table(df, values='score', index=['hole'], columns=['flag_loc'], aggfunc=(np.mean, np.std)).round(2)
            hole_prox = pd.pivot_table(df, values='prox', index=['stroke'], columns=['flag_loc', 'score'], aggfunc=(np.mean, np.std)).round(0).fillna("--")

        except Exception as e:
            score_avg = pd.DataFrame({})
            hole_prox = pd.DataFrame({})
        
        score_out.clear_output()
        display(
            widgets.Label(value="Hole & Pin Scoring"), score_avg, 
            widgets.Label(value="Stroke to Score Proximity"), hole_prox,
        )
    with img_out:
        hole_path = s.get_hole_img_path(Hole)
        if hole_path is not None:
            file = open(hole_path, "rb")
            image = file.read()
            hole_img = widgets.Image(
                value=image,
                format='jpeg',
            )
            img_out.clear_output()
            display(hole_img)
        
    with graph_out:
        if plot_bool:
            fig = pty
            fig.data = []
        else:
            fig = ax
            fig.cla()
        
        s.set_base_df(Hole, Flag_Loc)

        if Include_Flag_Locs:
#             try:
            fig = s.plot_flag_loc(fig, Hole, plotly=plot_bool)
#             except:
#                 pass

        if Include_Flag_Prox:
            fig = s.plot_shot_proximity(fig, Hole, Stroke, None, Round, Flag_Loc, plotly=plot_bool) 
            

        for score_val in [Score1, Score2, Score3]:
            if Include_Shots:
                try:
                    fig = s.plot_hole(fig, Hole, Stroke, score_val, Round, Flag_Loc, plotly=plot_bool)
                except:
                    pass

            if score_val is None:
                continue
            try:
                fig = s.plot_hole_distribution(fig, Hole, Stroke, score_val, Round, Flag_Loc, N=50, plotly=plot_bool)
            except:
                pass
            
            if Include_Spray:
                try:
                    fig = s.plot_shot_distribution(fig, Hole, Stroke, score_val, Round, Flag_Loc, plotly=plot_bool) 
                except:
                    pass


        graph_out.clear_output()
        if plot_bool:
            fig.layout.mapbox.center["lat"] = s.as_df(Hole).lat.mean()
            fig.layout.mapbox.center["lon"] = s.as_df(Hole).lon.mean()
            display(fig)   
        else:
            fig.grid()
            fig.axis('equal')
            display(mpl.canvas)
    
    return graph_out, score_out

hole=widgets.Dropdown(
    options=holes,
    description='Hole:',
    disabled=False,
    value=1,
)

score1=widgets.Dropdown(
    options=scores,
    description='Score 1:',
    disabled=False,        
    value=None,
)

score2=widgets.Dropdown(
    options=scores,
    description='Score 2:',
    disabled=False,
    value=None,
)

score3=widgets.Dropdown(
    options=scores,
    description='Score 3:',
    disabled=True,
    value=None,
)
stroke=widgets.Dropdown(
    options=strokes,
    description='Stroke:',
    disabled=False,
    value='all',
)

t_round=widgets.Dropdown(
    options=rounds,
    description='Round:',
    disabled=True,
    value='all',
)

flag=widgets.Dropdown(
    options=flags,
    description='Flag Position:',
    disabled=False,
    value='all',
)

inc_hole = widgets.Checkbox(
    value=True,
    description='Include Hole',
    disabled=False
)

inc_flag = widgets.Checkbox(
    value=False,
    description='Include Flag Locations',
    disabled=False
)

inc_prox = widgets.Checkbox(
    value=False,
    description='Include 95% Proximity',
    disabled=False
)

inc_spread = widgets.Checkbox(
    value=False,
    description='Include Spread',
    disabled=False
)

quick_render = widgets.Checkbox(
    value=use_plotly,
    description='Quick Render (No Map)',
    disabled=not use_plotly
)

hole_data = widgets.interactive_output(
    get_hole_data, 
    {
        "Hole": hole, 
        "Stroke": stroke, 
        "Round": t_round, 
        "Flag_Loc": flag, 
        "Score1": score1, 
        "Score2": score2, 
        "Score3": score3, 
        "Include_Shots": inc_hole,
        "Include_Flag_Locs": inc_flag,
        "Include_Flag_Prox": inc_prox,
        "Include_Spray": inc_spread,
        "Quick_Render": quick_render,
    }
)

input_boxes = widgets.VBox([quick_render, hole, stroke, t_round, flag, score1, score2, score3, inc_hole, inc_flag, inc_prox, inc_spread])

### Tournament

In [17]:
display(widgets.HBox(tourn_data.children[:-1]))

### Data

In [18]:
display(widgets.HBox([graph_out, input_boxes]))

In [19]:
display(img_out)

Output(layout=Layout(border='1px solid black', width='100%'), outputs=({'output_type': 'display_data', 'data':…

In [20]:
display(score_out)

Output(layout=Layout(border='1px solid black'), outputs=({'output_type': 'display_data', 'data': {'text/plain'…

IndexError: list index out of range

(1, 'all')

In [11]:
s._ShotDistribution__course_par

{}

In [42]:
import os
hole=1
hole_file_name = "0{}_full.jpg".format(hole)
        
ROOT_DIR = os.getenv("ROOT_DATA_DIR", "./third_party/pga-golf-data/data/")
tourn_id = s._ShotDistribution__get_tournament_id(s.name)
tourn_dir = ROOT_DIR + "tournaments/{}/".format(tourn_id)
for year in [2015, 2016, 2017]:
    year_dir = tourn_dir + "{}/".format(year) 
    course_dir = year_dir + "course/" + tourn_id + "/holes/"
    hole_dir = course_dir + hole_file_name
    print(os.path.exists(hole_dir))

True
True
True


In [38]:
file = open(hole_dir, "rb")
image = file.read()
widgets.Image(
    value=image,
    format='jpeg',
    width=300,
    height=400,
)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x1c\x00\x1c\x00\x00\xff\xdb\x00C\x00\x03\x02\x0…

In [39]:
max(['a', 1, 2, 4])

TypeError: '>' not supported between instances of 'int' and 'str'